In [ ]:
%pip install lightning transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import lightning as L
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.read_csv("./datasetino resumino/dataset_random_12b.csv", header=None, names=["job_offer", "cv", "score"])
print(df.head(6))

                                           job_offer  \
0  ## Business Analyst - Streamline Solutions\n\n...   
1  ## Python Developer - Join Our Growing Team!\n...   
2  ## Database Administrator\n\nWe are seeking a ...   
3  ## Python Developer\n\nWe are seeking a talent...   
4  ## Sales Representative - Drive Growth with In...   

                                                  cv  score  
0  **Eleanor Vance**\n(123) 456-7890 | eleanor.va...     68  
1  ## Amelia Stone\n(07700 900000) | amelia.stone...     12  
2  **Eliza Bloom**\n(123) 456-7890 | eliza.bloom@...     12  
3  ## Anya Sharma\n(123) 456-7890 | anya.sharma@e...     12  
4  ## Alex Chen\n(123) 456-7890 | alex.chen@email...     35  


In [6]:
df.dropna(inplace=True)
df['score'] = df['score'].astype(float) / 100.0

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
class CVJobDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=256):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        encoded = self.tokenizer(
            row['cv'],
            row['job_offer'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded['input_ids'].squeeze(0),
            'attention_mask': encoded['attention_mask'].squeeze(0),
            'label': torch.tensor(row['score'], dtype=torch.float)
        }


In [9]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

train_ds = CVJobDataset(train_df, tokenizer)
val_ds = CVJobDataset(val_df, tokenizer)
test_ds = CVJobDataset(test_df, tokenizer)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=4)
test_loader = DataLoader(test_ds, batch_size=4)

In [10]:
class CVJobMatchingModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.bert = AutoModel.from_pretrained("bert-base-uncased")
        self.regressor = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        self.loss_fn = nn.MSELoss()
        self.test_losses = []

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        return self.regressor(pooled_output).squeeze()

    def training_step(self, batch, batch_idx):
        preds = self(batch['input_ids'], batch['attention_mask'])
        labels = batch['label']
        loss = self.loss_fn(preds, labels)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        preds = self(batch['input_ids'], batch['attention_mask'])
        loss = self.loss_fn(preds, batch['label'])
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        preds = self(batch['input_ids'], batch['attention_mask'])
        labels = batch['label']
        loss = self.loss_fn(preds, labels)
        self.test_losses.append(loss.detach())

        preds_list = preds.detach().cpu().numpy().tolist()
        labels_list = labels.detach().cpu().numpy().tolist()

        if not isinstance(preds_list, list):
            preds_list = [preds_list]
        if not isinstance(labels_list, list):
            labels_list = [labels_list]

        self.y_true.extend(labels_list)
        self.y_pred.extend(preds_list)

        self.log("test_loss", loss)
        return loss

    def on_test_epoch_start(self):
        self.test_losses = []
        self.y_true = []
        self.y_pred = []

    def on_test_epoch_end(self):
        y_true_scaled = [y * 100 for y in self.y_true]
        y_pred_scaled = [y * 100 for y in self.y_pred]

        mse = mean_squared_error(y_true_scaled, y_pred_scaled)
        mae = mean_absolute_error(y_true_scaled, y_pred_scaled)
        r2 = r2_score(y_true_scaled, y_pred_scaled)

        print(f"\n📊 Test metrics (scaled to 0–100):")
        print(f"MSE: {mse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"R²: {r2:.4f}")

        metrics = {'MSE': mse, 'MAE': mae, 'R²': r2}
        names = list(metrics.keys())
        values = list(metrics.values())

        plt.figure(figsize=(6, 4))
        bars = plt.bar(names, values, color=['skyblue', 'orange', 'green'])
        plt.title("Regression Metrics on Test Set")
        plt.ylabel("Value")
        plt.ylim(0, max(values) * 1.2)

        for bar in bars:
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, f"{yval:.2f}", ha='center', va='bottom')

        plt.tight_layout()
        plt.show()

        errors = np.array(y_true_scaled) - np.array(y_pred_scaled)
        plt.figure(figsize=(10,5))
        sns.histplot(errors, bins=30, kde=True)
        plt.title("Histogram błędów predykcji (y_true - y_pred)")
        plt.xlabel("Błąd")
        plt.ylabel("Liczba próbek")
        plt.show()

        plt.figure(figsize=(8,8))
        plt.scatter(y_true_scaled, y_pred_scaled, alpha=0.5)
        plt.plot([0, 100], [0, 100], 'r--')
        plt.xlabel("Wartości prawdziwe")
        plt.ylabel("Wartości przewidywane")
        plt.title("Scatter plot: prawdziwe vs. przewidywane")
        plt.xlim(0, 100)
        plt.ylim(0, 100)
        plt.grid(True)
        plt.show()

        mean_vals = (np.array(y_true_scaled) + np.array(y_pred_scaled)) / 2
        diff_vals = np.array(y_true_scaled) - np.array(y_pred_scaled)
        mean_diff = np.mean(diff_vals)
        std_diff = np.std(diff_vals)

        plt.figure(figsize=(8,6))
        plt.scatter(mean_vals, diff_vals, alpha=0.5)
        plt.axhline(mean_diff, color='red', linestyle='--')
        plt.axhline(mean_diff + 1.96*std_diff, color='gray', linestyle='--')
        plt.axhline(mean_diff - 1.96*std_diff, color='gray', linestyle='--')
        plt.xlabel("Średnia (prawdziwe i przewidywane)")
        plt.ylabel("Różnica (prawdziwe - przewidywane)")
        plt.title("Bland-Altman plot")
        plt.grid(True)
        plt.show()

        def group_to_20_classes(values):
            values = np.clip(values.astype(int), 0, 100)
            return (values // 5).astype(int)

        label_20 = group_to_20_classes(np.array(y_true_scaled).round())
        pred_20 = group_to_20_classes(np.array(y_pred_scaled).round())

        cm_20 = confusion_matrix(label_20, pred_20, labels=range(20))

        plt.figure(figsize=(12,10))
        sns.heatmap(cm_20, annot=True, fmt='d', cmap='coolwarm')
        plt.title("Heatmapa błędów - grupowanie do 20 klas")
        plt.xlabel("Przewidywana klasa")
        plt.ylabel("Prawdziwa klasa")
        plt.show()


    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=2e-5)

In [ ]:
model = CVJobMatchingModel()
trainer = L.Trainer(max_epochs=9, accelerator='auto', devices='auto')
trainer.fit(model, train_loader, val_loader)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: 
  | Name      | Type       | Params | Mode 
-------------------------------------------------
0 | bert      | BertModel  | 109 M  | eval 
1 | regressor | Sequential | 98.6 K | train
2 | loss_fn   | MSELoss    | 0      | train
---------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
trainer.test(model, test_loader)